# Eliminación de preguntas repetidas
Ya con todas las preguntas que cumplen los requisitos del INE para ser tomadas en cuenta, vamos 
a hacer otra depuración para eliminar aquellas preguntas que están repetidas.
Es importante considerar que dos preguntas pueden ser iguales aún cuando estén formuladas con 
palabras diferentes (es decir, una puede ser paráfrasis de la otra).
El resultado de este paso serán dos archivos:

- Uno que contenga el texto de las preguntas repetidas, así como una métrica que mida su parecido. De estas preguntas, sólo se conservará 1 en el dataset de trabajo.

  - La pregunta que será conservada es aquella que fue enviada primero de acuerdo al timestamp
          
- Un archivo con las preguntas depuradas (eliminando las repeticiones)

In [12]:
!pip install spacy
!pip install scikit-learn
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     --------------------------------------- 0.0/42.3 MB 217.9 kB/s eta 0:03:14
     --------------------------------------- 0.1/42.3 MB 327.7 kB/s eta 0:02:09
     --------------------------------------- 0.1/42.3 MB 595.3 kB/s eta 0:01:11
     ---------------------------------------- 0.3/42.3 MB 1.4 MB/s eta 0:00:31
      --------------------------------------- 0.6/42.3 MB 2.4 MB/s eta 0:00:18
     - -------------------------------------- 1.4/42.3 MB 4.4 MB/s eta 0:00:10
     -- ------------------------------------- 2.7/42.3 MB 7.4 MB/s eta 0:00:06
     --- ------------------------------------ 4.2/42.3 MB 10.6 MB/s eta 0:00:04
     ----- ---------------------------------- 6.2/42.3 MB 14.1 MB/s eta 0:00:03
     ------- -------------------------------- 8.3/42.3 MB 16.5 MB/s eta 0:00:03
     --------- ----------------------------- 10.3/42.3 MB 26

In [18]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity

limpio_df = pd.read_csv('./Base_limpia.csv')

nlp = spacy.load('es_core_news_md')

preguntas = limpio_df['pregunta_norm']

vectores = [nlp(str(pregunta)).vector for pregunta in preguntas]

similitud = cosine_similarity(vectores)

repetidas = []

for index, fila in limpio_df.iterrows():
    for j in range(index + 1, len(preguntas)):
        if similitud[index, j] > 0.9:
            p1_fecha = limpio_df.loc[index]['fecha']
            p2_fecha = limpio_df.loc[j]['fecha']
            if p1_fecha < p2_fecha:
              repetidas.append(pd.DataFrame([[limpio_df.iloc[j]['pregunta'], limpio_df.loc[index]['pregunta']]], index=None))
            else:
              repetidas.append(pd.DataFrame([[limpio_df.loc[index]['pregunta'], limpio_df.iloc[j]['pregunta']]], index=None))
repetidas = pd.concat(repetidas)
repetidas.columns = ['repetida_de', 'pregunta']

display(repetidas)
repetidas.to_csv('./preguntas_repetidas.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 202851 entries, 0 to 0
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   repetida_de  202851 non-null  object
 1   pregunta     202851 non-null  object
dtypes: object(2)
memory usage: 4.6+ MB


(202851, 2)

Index(['repetida_de', 'pregunta'], dtype='object')

,repetida_de,pregunta
0,En México muchas personas se quejan del sistem...,Cómo podría usted cambiar el sistema de salud ...
0,En México muchas personas se quejan del sistem...,¿Cómo podrías incrementar la eficacia de los ...
0,En México muchas personas se quejan del sistem...,Que nos digan su opinión con respecto al servi...
0,En México muchas personas se quejan del sistem...,El sistema de salud es insuficiente mi pregunt...
0,En México muchas personas se quejan del sistem...,¿Cuál es su plan concreto para mejorar el acce...
...,...,...
0,¿Cuál sería su enfoque para abordar la violenc...,"Como candidato presidencial, ¿cuáles son sus p..."
0,¿Cómo abordará el problema de la desigualdad e...,¿Cómo abordará la brecha educativa entre zonas...
0,¿Cuál es su enfoque para abordar la violencia ...,¿Cuál es la estrategia proponen para prevenir ...
0,"¿Cuáles son sus propuestas para concientizar, ...",¿Qué estrategias implementarán para sensibiliz...


,pregunta
repetida_de,
!Que políticas gubernamentales se aplicarán en las escuelas para frenar la violencia en contra de las mujeres?,¿Cuáles serían sus estrategias para reducir la...
""";Cuál va hacer la estrategia para garantizar el abasto de salud pública y acabar con el actual desabasto de medicamentos ?",Que estrategias va a implementar para asegurar...
"""Candidata, en relación a la educación, ¿cómo planea garantizar la transparencia en la distribución de becas a estudiantes para asegurar que lleguen a aquellos que más lo necesitan y que el proceso sea justo y equitativo?""",La educación inclusiva es necesario implementa...
"""Candidatos, ante la corrupción en nuestro sistema judicial y el aumento de la violencia de género, ¿qué proponen para erradicar la corrupción e impunidad? Además, ¿cómo garantizarán que las autoridades no violen, a través de sobornos, las leyes aprobadas para proteger a las mujeres?""",Se habla de las deficiencias en el sistema de ...
"""Considerando la corrupción arraigada en el poder judicial, fiscalías y ministerios públicos, ¿qué medidas específicas propondrían para garantizar la liberación de individuos injustamente encarcelados debido a la fabricación de delitos y la corrupción institucional en el sistema de justicia mexicano",¿Cuál será su plan de acción para erradicar la...
...,...
"⁠¿Cuál es su programa de atención personas migrantes, tanto para aquellas que están en tránsito como para quienes desean residir en México?",¿Cuál es su programa de atención personas migr...
𝑷𝒐𝒓𝒒𝒖𝒆 𝒑𝒓𝒆𝒔𝒊𝒅𝒆𝒏𝒕𝒆𝒔 𝒚 𝒅𝒆𝒍𝒆𝒈𝒂𝒅𝒐𝒔 𝒅𝒆 𝒄𝒐𝒎𝒖𝒏𝒊𝒅𝒂𝒅𝒆𝒔 𝒏𝒐 𝒉𝒂𝒄𝒆𝒏 𝒏𝒂𝒅𝒂 𝒑𝒂𝒓𝒂 𝒍𝒂 𝒎𝒆𝒋𝒐𝒓𝒂 𝒅𝒆 𝒄𝒂𝒓𝒓𝒆𝒕𝒆𝒓𝒂𝒔 𝒚 𝒏𝒖𝒏𝒄𝒂 𝒂𝒏 𝒎𝒆𝒕𝒊𝒅𝒐 𝒂𝒍𝒖𝒎𝒃𝒓𝒂𝒅𝒐 𝒆𝒏 𝒄𝒂𝒓𝒓𝒆𝒕𝒆𝒓𝒂𝒔 𝒑𝒓𝒊𝒏𝒄𝒊𝒑𝒂𝒍𝒆𝒔 𝒅𝒆 𝒍𝒂 𝒄𝒐𝒎𝒖𝒏𝒊𝒅𝒂𝒅,𝟷𝟶𝟶% 𝙰𝚖𝚕𝚘 ❤No𝘈𝘺𝘶𝘥𝘢𝘳 𝘺 𝘢𝘱𝘰𝘺𝘢𝘳 𝘢 𝘭𝘢𝘴 𝘧𝘢𝘮𝘪𝘭𝘪𝘢𝘴 𝘪𝘮...
𝘈𝘺𝘶𝘥𝘢𝘳 𝘺 𝘢𝘱𝘰𝘺𝘢𝘳 𝘢 𝘭𝘢𝘴 𝘧𝘢𝘮𝘪𝘭𝘪𝘢𝘴 𝘪𝘮𝘪𝘨𝘳𝘢𝘯𝘵𝘦𝘴,Para ksNo seNadaNada.
